In [1]:
# import library

import torch
import numpy as np
import sys
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn

In [2]:
# check gpu 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [3]:
# Input

X_train = np.array([[73 , 67 , 43 ],
                    [91 , 88 , 64 ],
                    [87 , 134, 58 ],
                    [102, 43 , 37 ],
                    [69 , 96 , 70 ],
                    [73 , 67 , 43 ],
                    [91 , 88 , 64 ],
                    [87 , 134, 58 ],
                    [102, 43 , 37 ],
                    [69 , 96 , 70 ],
                    [73 , 67 , 43 ],
                    [91 , 88 , 64 ],
                    [87 , 134, 58 ],
                    [102, 43 , 37 ],
                    [69 , 96 , 70 ]], dtype='float32')

# Target

y_train = np.array([[56 , 70 ],
                    [81 , 101],
                    [119, 133],
                    [22 , 37 ],
                    [103, 119],
                    [56 , 70 ],
                    [81 , 101],
                    [119, 133],
                    [22 , 37 ],
                    [103, 119], 
                    [56 , 70 ],
                    [81 , 101],
                    [119, 133],
                    [22 , 27 ],
                    [103, 119]], dtype='float32')

# convert into tensor

inputs = torch.from_numpy(X_train)
target = torch.from_numpy(y_train)
print("input shape :", inputs.size())
print("target shape", target.size())

input shape : torch.Size([15, 3])
target shape torch.Size([15, 2])


In [4]:
# define dataset

train_ds = TensorDataset(inputs, target)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [5]:
# define data loader

batch_size = 3
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [6]:
# define model

model = nn.Linear(3, 2) #nn.linear assume this shape (in_features, out_features)

In [7]:
# define loss function

criterion_mse = nn.MSELoss()

# define optimizer

opt = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [8]:
# train function

def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # record loss for plot
    loss_history = []
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for inputs, target in train_dl:
            
            inputs.to(device)  # move to gpu if possible, it not, cpu
            target.to(device)
            
            # 1. Predict
            pred = model(inputs)
            
            # 2. Calculate loss
            loss = loss_fn(pred, target)
            # adding loss into the list
            loss_history.append(loss)
            
            # 3. Calculate gradient
            opt.zero_grad() # if not, gradients will accumulate
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            sys.stdout.write("\rEpoch [{}/{}], loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))

In [9]:
# train for 100 epochs
fit(100, model, criterion_mse, opt, train_dl)

Epoch [100/100], loss: 30427.5996

In [10]:
# Generate predictions

preds = model(inputs)
loss = criterion_mse(preds, target)
print(loss.item())

36872.20703125


In [ ]:
p